In [1]:
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
%pwd

'/home/anthony/new_all-code/Chatbot/research'

In [4]:
# Extract data from the PDF file
from pathlib import Path

def load_pdf_file(data):
    data_path = Path(data)
    if not data_path.exists():
        raise ValueError(f"Path {data_path} does not exist!")
        
    loader = DirectoryLoader(
        str(data_path),  # Convert to string for compatibility
        glob="*.pdf",
        loader_cls=PyMuPDFLoader
    )
    return loader.load()

In [5]:
def load_pdf_file_2(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyMuPDFLoader
    )

    document = loader.load()

    return document

In [7]:
from pathlib import Path

# Get the current directory and construct the Data path
data_dir = Path.cwd() / "Data"
print(f"Looking for PDFs in: {data_dir}")

# Verify the directory exists
if not data_dir.exists():
    raise FileNotFoundError(f"Data directory not found at: {data_dir}")

extracted_data = load_pdf_file(data=data_dir)

Looking for PDFs in: /home/anthony/new_all-code/Chatbot/research/Data


In [8]:
extract_df = load_pdf_file_2(data=data_dir)

In [10]:
extract_df

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2023-11-07T12:25:14+03:00', 'source': '/home/anthony/new_all-code/Chatbot/research/Data/Systems Security.pdf', 'file_path': '/home/anthony/new_all-code/Chatbot/research/Data/Systems Security.pdf', 'total_pages': 23, 'format': 'PDF 1.5', 'title': 'Systems Security', 'author': 'Microsoft account', 'subject': '', 'keywords': '', 'moddate': '2023-11-07T12:25:14+03:00', 'trapped': '', 'modDate': "D:20231107122514+03'00'", 'creationDate': "D:20231107122514+03'00'", 'page': 0}, page_content='Systems Security'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2023-11-07T12:25:14+03:00', 'source': '/home/anthony/new_all-code/Chatbot/research/Data/Systems Security.pdf', 'file_path': '/home/anthony/new_all-code/Chatbot/research/Data/Systems Security.pdf', 'total_pages': 23, 'format': 'PDF 1.5', 'title': 

In [11]:
# extracted_data

In [12]:
# Spliting the data into test chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk

In [14]:
text_chunk = text_split(extract_df)
print(f"Lenght of text extracted {len(text_chunk)}")

Lenght of text extracted 43


In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [32]:
# Download the embedding from hugging face
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [33]:
embeddings = download_hugging_face_embedding()

In [35]:
quert_reslts = embeddings.embed_query("Hello world")
print("Length: ", len(quert_reslts))

Length:  384


In [36]:
quert_reslts

[-0.034477315843105316,
 0.031023172661662102,
 0.006734910886734724,
 0.02610892429947853,
 -0.03936195746064186,
 -0.1603025197982788,
 0.06692396104335785,
 -0.006441440898925066,
 -0.04745054617524147,
 0.014758836477994919,
 0.07087532430887222,
 0.055527545511722565,
 0.01919332519173622,
 -0.026251299306750298,
 -0.01010951679199934,
 -0.026940451934933662,
 0.022307397797703743,
 -0.022226639091968536,
 -0.1496926248073578,
 -0.01749303936958313,
 0.007676327601075172,
 0.054352276027202606,
 0.0032544792629778385,
 0.03172592446208,
 -0.08462144434452057,
 -0.029405953362584114,
 0.05159562826156616,
 0.048124104738235474,
 -0.003314818488433957,
 -0.05827919766306877,
 0.04196928068995476,
 0.02221069671213627,
 0.12818878889083862,
 -0.02233896404504776,
 -0.011656257323920727,
 0.06292840093374252,
 -0.03287629410624504,
 -0.09122602641582489,
 -0.031175386160612106,
 0.05269954726099968,
 0.047034841030836105,
 -0.08420310169458389,
 -0.030056146904826164,
 -0.020744822919

In [39]:
from dotenv import load_dotenv
import os

load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [40]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [50]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
pc = Pinecone(api_key="your_api_key")  # Replace with your actual API key

# Create an index if it doesn't exist
if "chatbot" not in pc.list_indexes():
    pc.create_index(
        name="chatbot",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Connect to the index
index = pc.Index("chatbot")

print("Pinecone index setup complete!")



ImportError: cannot import name 'Pinecone' from 'pinecone' (unknown location)